## Principais Métricas:
Faturamento anual: Qual a faixa de faturamento anual da empresa? \
Tipo de carga: Que tipo de carga os clientes movimentam com mais frequência? \
Localização geográfica: Em qual região geográfica a maior parte dos clientes estão localizados? \
Tempo de parceria: Quais são os 4 clientes mais antigos? \
Empresa que mais investiu na Logmar: Qual empresa realizou maior investimento na empresa? \
Qual empresa menos investiu na Logmar?

In [65]:
#Aqui eu crio o DataFrame

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Logmar Clientes").getOrCreate()

df = spark.read.csv("C:/Users/gabri/Documents/LogMar/LogmarClientes.csv", header=True, inferSchema=True)

In [66]:
df.printSchema()

root
 |-- Código Cliente: string (nullable = true)
 |-- Nome Empresa: string (nullable = true)
 |-- Pagamento Anual: double (nullable = true)
 |-- Tipo de Carga: string (nullable = true)
 |-- Localização: string (nullable = true)
 |-- Início Parceria: string (nullable = true)



In [67]:
df.show()

+--------------+----------------+---------------+--------------+-------------------+---------------+
|Código Cliente|    Nome Empresa|Pagamento Anual| Tipo de Carga|        Localização|Início Parceria|
+--------------+----------------+---------------+--------------+-------------------+---------------+
|     CLNT23784|  TransLogística|      3700000.0|   Contêineres|          São Paulo|      4/12/2023|
|     CLNT82456|     TransEnvios|      1900000.0| Granel Sólido|     Rio de Janeiro|      9/30/2022|
|     CLNT39102|         LogCity|      2600000.0|   Contêineres|             Paraná|      11/5/2023|
|     CLNT14923|      MegaCargas|      1400000.0|Granel Líquido|              Bahia|      7/20/2023|
|     CLNT50761| SuperTransporte|      4300000.0|   Contêineres|     Santa Catarina|      2/28/2023|
|     CLNT62481|          LogFax|      2000000.0| Granel Sólido|       Minas Gerais|     10/14/2022|
|     CLNT89572|     GlobalFrete|      3500000.0|   Contêineres|  Rio Grande do Sul|       

In [68]:
# Aqui altero inicio parceria de string para date

from pyspark.sql.functions import to_date

df = df.withColumn('Início Parceria', to_date(df['Início Parceria'], 'M/d/yyyy'))

In [69]:
df.printSchema()

root
 |-- Código Cliente: string (nullable = true)
 |-- Nome Empresa: string (nullable = true)
 |-- Pagamento Anual: double (nullable = true)
 |-- Tipo de Carga: string (nullable = true)
 |-- Localização: string (nullable = true)
 |-- Início Parceria: date (nullable = true)



In [70]:
df.show()

+--------------+----------------+---------------+--------------+-------------------+---------------+
|Código Cliente|    Nome Empresa|Pagamento Anual| Tipo de Carga|        Localização|Início Parceria|
+--------------+----------------+---------------+--------------+-------------------+---------------+
|     CLNT23784|  TransLogística|      3700000.0|   Contêineres|          São Paulo|     2023-04-12|
|     CLNT82456|     TransEnvios|      1900000.0| Granel Sólido|     Rio de Janeiro|     2022-09-30|
|     CLNT39102|         LogCity|      2600000.0|   Contêineres|             Paraná|     2023-11-05|
|     CLNT14923|      MegaCargas|      1400000.0|Granel Líquido|              Bahia|     2023-07-20|
|     CLNT50761| SuperTransporte|      4300000.0|   Contêineres|     Santa Catarina|     2023-02-28|
|     CLNT62481|          LogFax|      2000000.0| Granel Sólido|       Minas Gerais|     2022-10-14|
|     CLNT89572|     GlobalFrete|      3500000.0|   Contêineres|  Rio Grande do Sul|     20

## Qual Faturamento Anual?

In [71]:
from pyspark.sql.functions import sum

faturamentoAnual = df.groupBy().agg(sum('Pagamento Anual'))
faturamentoTotal = faturamentoAnual.collect()[0][0]

print('O Faturamento Anual da Logmar é de: R$', faturamentoTotal)

O Faturamento Anual da Logmar é de: R$ 79100000.0


## Qual o tipo de carga mais frequente?

In [72]:
from pyspark.sql.functions import desc

tipoCargaMaisFrequente = df.groupBy('Tipo de Carga').count().orderBy(desc('count')).first()

print('O tipo de carga mais frequente é:', tipoCargaMaisFrequente['Tipo de Carga'])

O tipo de carga mais frequente é: Contêineres


## Localização da maior parte dos clientes?

In [73]:
regiaoMaisClientes = df.groupBy('Localização').count().orderBy(desc('count'))

regiaoMaisRepetida = df.groupBy('Localização').count().orderBy(desc('count')).first()
print(regiaoMaisRepetida['Localização'], 'é a região com mais clientes.')

Espírito Santo é a região com mais clientes.


## Quais são os 4 clientes mais antigos?

In [74]:
from pyspark.sql.functions import asc

clientesMaisAntigos = df.orderBy(asc('Início Parceria')).limit(4)

print('Os quatro clientes mais antigos são:')
clientesMaisAntigos.select('Nome Empresa', 'Início Parceria').show()

Os quatro clientes mais antigos são:
+------------+---------------+
|Nome Empresa|Início Parceria|
+------------+---------------+
|  RapidPorto|     2022-07-30|
|     TransEx|     2022-09-15|
|  TransGetex|     2022-09-25|
| TransEnvios|     2022-09-30|
+------------+---------------+



## Qual empresa realizou maior investimento na Logmar?

In [75]:
from pyspark.sql.functions import col

maiorInvestimento = df.groupBy('Nome Empresa').agg({'Pagamento Anual': 'sum'}).orderBy(col('sum(Pagamento Anual)').desc()).first()

print('A empresa que mais investiu na Logmar foi a', maiorInvestimento['Nome Empresa'], 'com um total de: R$', maiorInvestimento['sum(Pagamento Anual)'])

A empresa que mais investiu na Logmar foi a SuperTransporte com um total de: R$ 4300000.0


## Qual empresa realizou menor investimento na Logmar?

In [76]:
menorInvestimento = df.groupBy('Nome Empresa').agg({'Pagamento Anual': 'sum'}).orderBy(col('sum(Pagamento Anual)').asc()).first()

print('A empresa que menos investiu na Logmar foi a', menorInvestimento['Nome Empresa'], 'com um total de: R$', menorInvestimento['sum(Pagamento Anual)'])

A empresa que menos investiu na Logmar foi a MegaCargas com um total de: R$ 1400000.0
